In [1]:
# Editor appearance set up & Load plot & Calculate index

# Change backend for interactive plot
%matplotlib nbagg
# %matplotlib osx
# %matplotlib tk
# %matplotlib qt

# This line of "print" must exist right after %matplotlib command, 
# otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')


# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Import packages needed

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.widgets import RectangleSelector, PolygonSelector
import numpy as np
from skimage import io, draw
import mplcursors
import IPython.display as Disp
from ipywidgets import widgets
import cv2
from osgeo import gdal
import pickle
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename


# Load image and print size & pre-process

# Use skimage to load multi-layer tiff file
Tk().withdraw()
image_file = askopenfilename(title='Load image file')
rgb_ir_img = io.imread(image_file)
print("Image Shape: ", rgb_ir_img.shape)

# Extract layers from the multilayer tiff file and do some adjustments
layer_RGB = np.copy(rgb_ir_img[:, :, 0:3])
layer_IR = np.copy(rgb_ir_img[:, :, 3])
layer_mask = np.copy(rgb_ir_img[:, :, 4])

# Change from 16 bit to 8 bit(max 65535 to max 255)
layer_RGB = np.round(layer_RGB / 257)
layer_RGB = layer_RGB.astype(int)

# Set background to black (You can choose background color when exporting orthomosaic from agisoft, 
# the default is white)
layer_RGB[np.where(layer_mask == 0)] = 0
layer_IR[np.where(layer_mask == 0)] = 0



# Load GPS coordinate from file & Calculate pixel location
plot_loc = askopenfilename(title='Load plot location file')
try:
    with open(plot_loc, 'rb') as f:
        plot_names = pickle.load(f)
        plot_vertices_gps = pickle.load(f)
except Exception as e:
    showerror(type(e).__name__, str(e))


# Calculating pixel location from GPS coordinate

ds = gdal.Open(image_file)
gt = ds.GetGeoTransform()
a, b, c, d, e, f = gt

plot_vertices = []
for i in range(len(plot_vertices_gps)):
    one_plot_vertices_gps = plot_vertices_gps[i]
    one_plot_vertices = []
    for vertex in one_plot_vertices_gps:
        xgeo, ygeo = vertex
        
        x = (xgeo - a) / b
        y = (ygeo - d) / f

        one_plot_vertices.append([x, y])
    one_plot_vertices = np.array(one_plot_vertices)
    one_plot_vertices = np.round(one_plot_vertices)
    one_plot_vertices = one_plot_vertices.astype(int)
    plot_vertices.append(one_plot_vertices)

print('Plot location loaded')



# RGB to HSV (H 0-1, S 0-1, V 0-255)

layer_HSV = matplotlib.colors.rgb_to_hsv(layer_RGB)

layer_hue = layer_HSV[:, :, 0]
layer_saturation = layer_HSV[:, :, 1]
layer_value = layer_HSV[:, :, 2]


# Calculate Vegetation Index and display

# Original formula
# DGCI = ((layer_HSV[:, :, 0] - 60) / 60 + (1 - layer_HSV[:, :, 1]) + (1 - layer_HSV[:, :, 2]/255)) / 3

layer_hue = layer_hue * 360
layer_hue = layer_hue - 60
layer_hue[np.where(layer_hue<0)] = 0
layer_hue[np.where(layer_hue>60)] = 60
print("Adjusted hue min: ", layer_hue.min())
print("Adjusted hue max: ", layer_hue.max())

layer_DGCI = (layer_hue / 60 + (1 - layer_saturation) + (1 - layer_value / 255)) / 3

# Adjust value to show
layer_DGCI[np.where(layer_mask == 0)] = layer_DGCI.min()

print('DGCI calculated')

Interactive plot activated


Image Shape:  (9171, 5093, 5)
Plot location loaded
Adjusted hue min:  0.0
Adjusted hue max:  60.0
DGCI calculated


In [ ]:
# Show indices of plots

def show_RGB(b):
    global cb
    if cb is not None:
        cb.remove()
        cb = None
    ax.imshow(interested_area_RGB)
    
def show_IR(b):
    global cb
    if cb is not None:
        cb.remove()
    axim = ax.imshow(interested_area_IR, cmap=plt.get_cmap(color_map))
    cb = fig.colorbar(axim)
    
def show_DGCI_image(b):
    global cb
    if cb is not None:
        cb.remove()
    axim = ax.imshow(interested_area_DGCI, cmap=plt.get_cmap(color_map))
    cb = fig.colorbar(axim)
    
def show_temp(b):
    if b.description == 'Show Average Temperature':
        for i in range(len(plot_names)):
            one_plot_vertices = plot_vertices[i]
            one_plot_vertices_transformed = one_plot_vertices - ul
            
            polygon = patches.Polygon(one_plot_vertices_transformed, True, facecolor = matplotlib.colors.to_rgba('red', 0.05), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
            ax.add_patch(polygon)
            text_loc = np.mean(one_plot_vertices_transformed, 0)
            axtx = ax.text(text_loc[0], text_loc[1], plot_names[i] + '\n' + str(avg_temps[i]) + '℃', ha='center', va='center')
            
        b.description = 'Hide Average Temperature'
    
    elif b.description == 'Hide Average Temperature':
        ax.patches.clear()
        ax.texts.clear()
        b.description = 'Show Average Temperature'
        plt.show()
    
def show_DGCI(b):
    if b.description == 'Show Average DGCI':
        for i in range(len(plot_names)):
            one_plot_vertices = plot_vertices[i]
            one_plot_vertices_transformed = one_plot_vertices - ul
            
            polygon = patches.Polygon(one_plot_vertices_transformed, True, facecolor = matplotlib.colors.to_rgba('red', 0.05), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
            ax.add_patch(polygon)
            text_loc = np.mean(one_plot_vertices_transformed, 0)
            axtx = ax.text(text_loc[0], text_loc[1], plot_names[i] + '\n' + str(avg_DGCIs[i]), ha='center', va='center')
            
        b.description = 'Hide Average DGCI'
    
    elif b.description == 'Hide Average DGCI':
        ax.patches.clear()
        ax.texts.clear()
        b.description = 'Show Average DGCI'
        plt.show()




color_map = 'gist_gray'
all_vertices = np.concatenate(plot_vertices, axis=0)
ul = np.amin(all_vertices, axis=0)
br = np.amax(all_vertices, axis=0)

interested_area_IR = layer_IR[ul[1]:br[1], ul[0]:br[0]]
interested_area_RGB = layer_RGB[ul[1]:br[1], ul[0]:br[0], :]
interested_area_DGCI = layer_DGCI[ul[1]:br[1], ul[0]:br[0]]

# Calculate avg temp & DGCI for each plot
avg_temps = []
avg_DGCIs = []
avg_hues = []
avg_sats = []
avg_vals = []
for i in range(len(plot_names)):
    one_plot_vertices = plot_vertices[i]
    one_plot_vertices_transformed = one_plot_vertices - ul
    
    rr, cc = draw.polygon(one_plot_vertices_transformed[:, 1], one_plot_vertices_transformed[:, 0], interested_area_IR.shape)
    mask = np.zeros(interested_area_IR.shape, np.float)
    mask[rr, cc] = 1
    inds = np.where(mask == 1)
    avg_IR = np.average(interested_area_IR[inds])
    avg_temp = np.round(avg_IR * 0.04 - 273.15, 2)
    avg_temps.append(avg_temp)
    
    avg_DGCI = np.average(interested_area_DGCI[inds])
    avg_DGCI = np.round(avg_DGCI, 2)
    avg_DGCIs.append(avg_DGCI)


button_layout = widgets.Layout(width='50%')
show_RGB_button = widgets.Button(description='Show RGB Image', layout=button_layout)
show_IR_button = widgets.Button(description='Show IR Image', layout=button_layout)
show_DGCIImage_button = widgets.Button(description='Show DGCI Image', layout=button_layout)
show_temp_button = widgets.Button(description='Show Average Temperature', layout=button_layout)
show_DGCI_button = widgets.Button(description='Show Average DGCI', layout=button_layout)

box_layout = widgets.Layout(align_items='center', width='50%')
buttons = widgets.VBox(children=[show_RGB_button, show_IR_button, show_DGCIImage_button, show_temp_button, show_DGCI_button], layout=box_layout)

out = widgets.Output()
all_widgets = widgets.HBox(children=[out, buttons])
display(all_widgets)

show_RGB_button.on_click(show_RGB)
show_IR_button.on_click(show_IR)
show_DGCIImage_button.on_click(show_DGCI_image)
show_temp_button.on_click(show_temp)
show_DGCI_button.on_click(show_DGCI)

with out:
    fig, ax = plt.subplots(figsize=(5, 5))
    fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9)
    ax.imshow(interested_area_RGB)
    cb = None
    plt.show()

In [ ]:
for i in range(len(plot_names)):
    print(plot_names[i])
    print('avg temp', avg_temps[i])
    print('avg DGCI', avg_DGCIs[i])
    print()